## TP1 d'Apprentissage Statistique 

### Binome:
- NKOUNGHAWE TOMEYUM Rosalie Corine
- LENO Celestine  

### A- Les données 

La fonction qui permet d'engendrer une matrice dont chaque valeur est une observation d'une personne avec 3 caractéristiques 

In [1]:
import random
import numpy as np 

def generer_tableau(n): # n est le nombre de lignes de la matrice, le nombre de colonnes est 3

    # Nous définissons les domaines de chaque observation, dans lesquels nous allons faire la sélection des valeurs 
    # pour former la matrice 

    cheveux = ['B', 'D']   # Blond, Dark
    hauteur = ['T', 'S']   # Tall, Short
    pays = ['G', 'P']      # Greenland, Poland
    
    tableau = []
    for _ in range(n):  # pour chaque ligne
        ligne = []
        for _ in range(3):  # 3 colonnes
            c = random.choice(cheveux)
            h = random.choice(hauteur)
            p = random.choice(pays)
            ligne.append((c, h, p))  # un triplet
        tableau.append(ligne)
    return tableau

# Générons une matrice un peu plus grande avec 100 lignes 
print(generer_tableau(100))


[[('B', 'S', 'P'), ('B', 'S', 'P'), ('D', 'S', 'P')], [('D', 'T', 'G'), ('B', 'S', 'P'), ('B', 'S', 'P')], [('D', 'T', 'G'), ('B', 'T', 'P'), ('D', 'T', 'G')], [('D', 'S', 'P'), ('B', 'S', 'P'), ('B', 'S', 'G')], [('B', 'S', 'G'), ('D', 'T', 'P'), ('B', 'S', 'P')], [('B', 'S', 'P'), ('D', 'T', 'G'), ('D', 'T', 'G')], [('D', 'T', 'G'), ('B', 'T', 'P'), ('B', 'T', 'G')], [('B', 'S', 'P'), ('B', 'S', 'P'), ('D', 'T', 'P')], [('D', 'T', 'G'), ('D', 'S', 'G'), ('B', 'S', 'G')], [('D', 'T', 'P'), ('D', 'S', 'G'), ('D', 'S', 'G')], [('B', 'S', 'P'), ('D', 'S', 'P'), ('D', 'T', 'P')], [('B', 'S', 'G'), ('D', 'S', 'G'), ('D', 'S', 'G')], [('D', 'T', 'P'), ('B', 'T', 'G'), ('D', 'S', 'G')], [('D', 'S', 'G'), ('D', 'T', 'P'), ('D', 'T', 'G')], [('D', 'T', 'P'), ('D', 'S', 'G'), ('D', 'S', 'G')], [('D', 'T', 'G'), ('B', 'T', 'P'), ('D', 'T', 'P')], [('D', 'T', 'G'), ('B', 'T', 'P'), ('D', 'T', 'G')], [('B', 'T', 'P'), ('D', 'S', 'P'), ('B', 'T', 'P')], [('B', 'S', 'G'), ('D', 'S', 'G'), ('D', 'S',

In [2]:
# La matrice que nous allons utiliser 

# Pour controler l'aléa 
random.seed(42)
data = generer_tableau(100)

### B- Questions : nationalité la plus probable d'un individu grand (T) et qui a les cheveux Blond (B)

Les fonctions que nous allons écrire dans la suite sont par rapport à la matrice que nous avons définie

### C- Maximum a posteriori
Nous allons utiliser deux versions de cette métrique pour conclure sur la question:  
- Une qui se fera avec naive bayes 
- Une autre qui ne tiendra pas compte de navive bayes : se basera uniquement sur les fréquences d'apparution, via la formule des probabilités conditionnelles

In [3]:
## Nous allons donc faire le calcul de probabilité avec plusieurs petites fonctions que nous allons utiliser
# par la suite dans une fonction générale 

##1- Version avec le théorème de bayes

"""
Il s'agit de calculer deux probabilités a posteriori et de prendre le maximum afin de conclure sur la nationalité de l'individu
Les formules sont les suivantes: 

P(G/B,T) = P(G)*P(B,T/G) / P(B,T)
P(P/B,T) = P(P)*P(B,T/P) / P(B,T)

Nous aurons besoin de caluler :
P(G): pg(), 
P(P): pp(), 
P(B,T/G): pbtg(), 
P(B,T/P): pbtp() 
et P(B,T): pbt()

P(G) = nombre de personnes de nationalité G / nombre total d'individus
P(P) = nombre de personnes de nationalité P / nombre total d'individus

P(B,T/G) = P(B/G)*P(T/G) via l'hypothèse d'indépendance de naive bayes
P(B/G) : pbg() = nombre de personnes aux cheveux Blonds / nombre de personnes de nationalité G
P(T/G) : ptg() = nombre de personnes de hauteur T / nombre de personnes de nationalité G

P(B,T/P) = P(B/P)*P(T/P) pareil que précédemment pour la nationalité G
P(B/P) : pbp() = nombre de personnes aux cheveux Blonds / nombre de personnes de nationalité P
P(T/P) : ptp() = nombre de personnes de hauteur T / nombre de personnes de nationalité P

"""


# -------------- Fonctions utilitaires -------------------- #


def pg(data):
    nb_G = 0
    nb_Total = len(data) * len(data[0])
    for ligne in data:
        for indiv in ligne:
            if indiv[2] == 'G':
                nb_G += 1
    return nb_G / nb_Total

def pp(data):
    nb_P = 0
    nb_Total = len(data) * len(data[0])
    for ligne in data:
        for indiv in ligne:
            if indiv[2] == 'P':
                nb_P += 1
    return nb_P / nb_Total

def pbg(data):
    nb_blond_G, nb_G = 0, 0
    for ligne in data:
        for indiv in ligne:
            if indiv[2] == 'G':
                nb_G += 1
                if indiv[0] == 'B':  # cheveux blonds
                    nb_blond_G += 1
    return nb_blond_G / nb_G if nb_G > 0 else 0

def ptg(data):
    nb_T_G, nb_G = 0, 0
    for ligne in data:
        for indiv in ligne:
            if indiv[2] == 'G':
                nb_G += 1
                if indiv[1] == 'T':
                    nb_T_G += 1
    return nb_T_G / nb_G if nb_G > 0 else 0

def pbp(data):
    nb_blond_P, nb_P = 0, 0
    for ligne in data:
        for indiv in ligne:
            if indiv[2] == 'P':
                nb_P += 1
                if indiv[0] == 'B':  # cheveux blonds
                    nb_blond_P += 1
    return nb_blond_P / nb_P if nb_P > 0 else 0

def ptp(data):
    nb_T_P, nb_P = 0, 0
    for ligne in data:
        for indiv in ligne:
            if indiv[2] == 'P':
                nb_P += 1
                if indiv[1] == 'T':
                    nb_T_P += 1
    return nb_T_P / nb_P if nb_P > 0 else 0

def pbtg(data):
    return pbg(data) * ptg(data)

def pbtp(data):
    return pbp(data) * ptp(data)

def pbt(data):
    # P(B,T) = somme des cas pour G et P
    # Nous appliquons la loi des proba totales, vu qu'on veut les individus 
    # avec des cheveux B et une hauteur T quelque soit la nationalité
    
    return pbtg(data) * pg(data) + pbtp(data) * pp(data)


# -------------------------Fonction générale------------------#


def classer_individu(data):
    # probas a posteriori
    p_G = (pg(data) * pbtg(data)) / pbt(data)
    p_P = (pp(data) * pbtp(data)) / pbt(data)

    if p_G > p_P:
        return f"Nationalité G avec une proba Max de {p_G}"
    elif p_P > p_G:
        return f"Nationalité P avec une proba Max de {p_P}"
    else:
        return "Indécis (égalité)"


In [4]:
classer_individu(data)

'Nationalité P avec une proba Max de 0.510296618174948'

### Conclusion sur la nationalité avec la première approche: 

Nous voyons que la probabilité de la nationalité P est celle trouvé comme plus grande 

## Reprenons la fonction cette fois sans utiliser le théorème de naive bayes

Il sera question maintenant de faire le calcul de proba conditionnelle

P(G/B,T) = P(B,T,G)/P(B,T) un peu comme la proba de l'intersection de G avec (B,T) divisé par P(B,T)

Ici il s'agit de compter le nombre de fois qu'on a un individu B, T, G et diviser par le nombre d'individu total ensuit diviser par le calul du nombre de personnes aux cheveux B et une hauteur T et diviser par le nombre total d'individus 

P(P/B,T): on fait pareil sur la nationalité P, on regarde le max et on conclu sur la nationalité 

In [5]:
# Fonctions utilitaires pour cette deuxième version qui n'utilise pas le théorème de Bayes 

"""
Ici, on calcule directement les probabilités par comptage dans le jeu de données.

Formules (probabilité conditionnelle par fréquence):

P(G / B,T) = P(B,T,G) / P(B,T) 
P(B,T,G) = nombre d'individus (B,T,G) / nombre total d'individus
P(B,T)   = nombre d'individus (B,T,*) / nombre total d'individus

P(P / B,T) = P(B,T,P) / P(B,T) 
P(B,T,P) = nombre d'individus (B,T,P) / nombre total d'individus
"""

def pbtg_v2(data):
    nb_BTG = 0
    nb_Total = len(data) * len(data[0])
    for ligne in data:
        for indiv in ligne: 
            if indiv[0] == 'B' and indiv[1] == 'T' and indiv[2] == 'G':
                nb_BTG += 1
    return nb_BTG / nb_Total if nb_Total > 0 else 0

def pbtp_v2(data):
    nb_BTP = 0
    nb_Total = len(data) * len(data[0])
    for ligne in data:
        for indiv in ligne: 
            if indiv[0] == 'B' and indiv[1] == 'T' and indiv[2] == 'P':
                nb_BTP += 1
    return nb_BTP / nb_Total if nb_Total > 0 else 0

def pbt_v2(data):
    nb_BT = 0
    nb_Total = len(data) * len(data[0])
    for ligne in data:
        for indiv in ligne: 
            if indiv[0] == 'B' and indiv[1] == 'T':  # Nationalité indifférente
                nb_BT += 1
    return nb_BT / nb_Total if nb_Total > 0 else 0

def classer_individu_v2(data):
    p_G = pbtg_v2(data) / pbt_v2(data) if pbt_v2(data) > 0 else 0
    p_P = pbtp_v2(data) / pbt_v2(data) if pbt_v2(data) > 0 else 0
    
    if p_G > p_P:
        return f"Nationalité G avec proba:  (p={p_G:.3f})"
    elif p_P > p_G:
        return f"Nationalité P avec proba: (p={p_P:.3f})"
    else:
        return f"Indécis (égalité, la proba est: p={p_G:.3f})"


In [6]:
classer_individu_v2(data)

'Nationalité P avec proba: (p=0.560)'

### Commentaire:

Avec notre jeu de données, on constate que les deux approches disent que l'individu est de nationalité P bien que nous remarquons que la probabilité avec la seconde approche semble être plus grande 

La différence entre les deux approches est que la première est plus stable, c'est moins probable d'avoir des 0 comme avec la deuxième approche. En effet, P(B,T/G) avec la deuxième approche vu qu'on calcul P(B,T,G), c'est parfois difficile d'avoir cette combinaison dans le jeu de donnée, ce qui n'est pas le cas lorsqu'on suppose l'indépendance entre les variables et qu'on applique la formule P(B/G)*P(T/G)

## D- Estimateur de Maximum de Vraisemblance : MLE

Il est question dans cette partie de trouver la classe qui maximise plutôt cette probabilité : P((B,T)/nationalité)

Donc *P(observation/classe)*:
- P((B,T)/P) : la nationalité P
- P((B,T)/G) : la nationalité G

Nous devons faire ceci en utilisant deux approches:
1. En tenant compte du cadre bayes naïf
P(B,T/G) = P(B/G)*P(T/G) comme donné dans l'énnoncé du TP pareil pour P(B,T/P)
2. Sans tenir compte du cadre bayes naïf

    Ici on fonctionnera par comptage dans notre matrice de données, comme avec l'exemple donné dans l'énnoncé du TP 


In [17]:
pbtg_sans_naif = pbg(data)*ptg(data)

print(pbtg_sans_naif)

0.24319759804841432


In [18]:
pbtp_sans_naif = pbp(data)*ptp(data)

print(pbtp_sans_naif)

0.24025974025974026


In [ ]:
# Le code pour répondre à la question avec la première approche est le suivant 


# Notre fonction est un peu plus générale, nous allons lui passer les données, la couleur des cheveux B et la hauteur T pour voir 
# la classe qui maximise la vraisemblance 
def mle_naif(data, cheveux, hauteur):
    # Comptage des individus par classe
    # On récupère de manière individuelle les classes présentes dans les données
    classes = set(indiv[2] for ligne in data for indiv in ligne)

    # où on pourra stocker les vraissemblances calculées et retenir la valeur maximale 
    vraisemblances = {}

    # rappel: le triplet (B,T,G) est dans l'ordre des caractéristiques (cheveux, hauteur et nationalité)
    
    for c in classes:
        nb_c = sum(indiv[2] == c for ligne in data for indiv in ligne)
        nb_cheveux_c = sum(indiv[2] == c and indiv[0] == cheveux for ligne in data for indiv in ligne)
        nb_hauteur_c = sum(indiv[2] == c and indiv[1] == hauteur for ligne in data for indiv in ligne)
        
        p_cheveux = nb_cheveux_c / nb_c if nb_c > 0 else 0
        p_hauteur  = nb_hauteur_c / nb_c if nb_c > 0 else 0
        
        vraisemblances[c] = p_cheveux * p_hauteur
    
    # Choisir la classe qui maximise P(x|c)
    best_class = max(vraisemblances, key=vraisemblances.get)
    return best_class, vraisemblances


In [8]:
## Test de notre fonction pour répondre à la question

print(f"La distribution de vraisemblance entre les classes est: {mle_naif(data=data, cheveux='B', hauteur='T')[1]} \nLa nationalité finale est: {mle_naif(data=data, cheveux='B', hauteur='T')[0]}")

La distribution de vraisemblance entre les classes est: {'G': 0.24319759804841432, 'P': 0.24025974025974026} 
La nationalité finale est: G


In [ ]:
## Si on utilisait nos fonctions définies plus haut, voici comment serait simplement le code 

def mle_naif_v2(data):

    # on calcul P(B,T/G) = P(B/G)*P(T/G) on l'appellera p1
    p1 = pbg(data)*ptg(data)

    # On calcul ensuite P(B,T/P) = P(B/P) * P(T/P) on l'appellera p2
    p2 = pbp(data)*ptp(data)
    
    # comparaison et choix de la nationalité 
    if p1 > p2:
        print(f"L'individu (B,T) est de nationalité G avec une vraisemblance de {p1}")
    else:
        print(f"L'individu (B,T) est de nationalité P avec une vraisemblance de {p2}")

In [21]:
## Application 
mle_naif_v2(data)

L'individu (B,T) est de nationalité G avec une vraisemblance de 0.24319759804841432


In [9]:
# Le code pour répondre à la question avec la deuxième approche est le suivant

def mle_sans_naif(data, cheveux, hauteur):
    classes = set(indiv[2] for ligne in data for indiv in ligne)
    vraisemblances = {}
    
    for c in classes:
        nb_c = sum(indiv[2] == c for ligne in data for indiv in ligne)
        nb_obs_c = sum(indiv[2] == c and indiv[0] == cheveux and indiv[1] == hauteur 
                       for ligne in data for indiv in ligne)
        
        vraisemblances[c] = nb_obs_c / nb_c if nb_c > 0 else 0
    
    best_class = max(vraisemblances, key=vraisemblances.get)
    return best_class, vraisemblances


In [10]:
# Application de la fonction avec le cas (B,T,G)

print(f"La distribution des proba entre les classes est: {mle_sans_naif(data=data, cheveux='B', hauteur='T')[1]} \nLa nationalité finale est: {mle_sans_naif(data=data, cheveux='B', hauteur='T')[0]}")

La distribution des proba entre les classes est: {'G': 0.22602739726027396, 'P': 0.2727272727272727} 
La nationalité finale est: P


## E- Point de vue continu

Dans ce cadre, on suppose que la mesure de la couleur des cheveux n’est plus binaire (B ou D) mais une variable continue notée intensité appartenant à [0,1] :

- 0 = blond pur
- 1 = dark pur
- Entre les deux, ce sont des valeurs intermédiaires.

1. Génération du jeu de données

Nous avons engender ce nouveau jeu de données avec la fonction qui génère aléatoirement des triplets (intensité, hauteur, pays)

In [11]:
def generer_tableau_continu(n):
    tableau = []
    for _ in range(n):
        intensite = random.uniform(0,1)  # valeur continue entre 0 et 1
        hauteur = random.choice(['T','S'])
        pays = random.choice(['G','P'])
        tableau.append((intensite, hauteur, pays))
    return tableau

# Générons pour 100 individus
data_continu = generer_tableau_continu(100)
data_continu

[(0.5719226923507389, 'S', 'P'),
 (0.021939597859565008, 'S', 'P'),
 (0.008097965761490244, 'T', 'P'),
 (0.8328388996731884, 'S', 'G'),
 (0.607064998164266, 'S', 'G'),
 (0.6366178780058919, 'S', 'G'),
 (0.6282481036868983, 'T', 'P'),
 (0.7886104840101539, 'T', 'P'),
 (0.7323597679392383, 'T', 'P'),
 (0.3267191257140104, 'S', 'G'),
 (0.2007947946500762, 'S', 'P'),
 (0.8304726195673974, 'S', 'P'),
 (0.9673840761668122, 'S', 'P'),
 (0.8042325386045106, 'S', 'G'),
 (0.14075701500588267, 'T', 'P'),
 (0.9672697473000323, 'S', 'G'),
 (0.7906819685889375, 'T', 'P'),
 (0.5365723082690591, 'S', 'P'),
 (0.6728283818737536, 'S', 'P'),
 (0.8224173012267743, 'S', 'G'),
 (0.674836555375322, 'S', 'G'),
 (0.6195468670445295, 'S', 'G'),
 (0.6476523806394415, 'S', 'P'),
 (0.6492394462845911, 'T', 'G'),
 (0.045346811013382116, 'T', 'P'),
 (0.9959273747550295, 'T', 'G'),
 (0.23480003243132908, 'T', 'P'),
 (0.4536437568888926, 'S', 'G'),
 (0.8854137838995794, 'T', 'P'),
 (0.6156522140511563, 'S', 'G'),
 (0.

2. Explication 

Pour répondre à la même question dans ce contexte, nous allons choisir un seuil (0.5 pour notre) :

- intensité < 0.5 sera considéré comme Blond (B)
- intensité > 0.5 sera considéré comme Dark (D)

In [12]:
def est_blond(intensite, seuil=0.5):
    return intensite < seuil

def pg_continu(data):
    nb_G = sum(1 for intensite, h, p in data if p == 'G')
    return nb_G / len(data)

def pp_continu(data):
    nb_P = sum(1 for intensite, h, p in data if p == 'P')
    return nb_P / len(data)

def pbg_continu(data, seuil=0.5):
    nb_blond_G, nb_G = 0, 0
    for intensite, h, p in data:
        if p == 'G':
            nb_G += 1
            if est_blond(intensite,seuil):
                nb_blond_G += 1
    return nb_blond_G / nb_G if nb_G > 0 else 0

def ptg_continu(data):
    nb_T_G, nb_G = 0, 0
    for intensite, h, p in data:
        if p == 'G':
            nb_G += 1
            if h == 'T':
                nb_T_G += 1
    return nb_T_G / nb_G if nb_G > 0 else 0

def pbp_continu(data, seuil=0.5):
    nb_blond_P, nb_P = 0, 0
    for intensite, h, p in data:
        if p == 'P':
            nb_P += 1
            if est_blond(intensite,seuil):
                nb_blond_P += 1
    return nb_blond_P / nb_P if nb_P > 0 else 0

def ptp_continu(data):
    nb_T_P, nb_P = 0, 0
    for intensite, h, p in data:
        if p == 'P':
            nb_P += 1
            if h == 'T':
                nb_T_P += 1
    return nb_T_P / nb_P if nb_P > 0 else 0
print("P(G) =", pg_continu(data_continu))
print("P(P) =", pp_continu(data_continu))
print("P(B|G) =", pbg_continu(data_continu))
print("P(T|G) =", ptg_continu(data_continu))
print("P(B|P) =", pbp_continu(data_continu))
print("P(T|P) =", ptp_continu(data_continu))

P(G) = 0.49
P(P) = 0.51
P(B|G) = 0.3673469387755102
P(T|G) = 0.40816326530612246
P(B|P) = 0.45098039215686275
P(T|P) = 0.5294117647058824


3. Modifications dans les fonctions Python

- Les fonctions comme pbg(data) devront être remplacées par une version qui compte selon un seuil d’intensité (ex. intensité < 0.5 = blond).

- Les fonctions de type pg(data), pp(data) restent identiques car elles ne dépendent pas de la continuité.

In [13]:
# Exemple de modification avec naïve Bayes des valeurs continues
def classer_individu_continu_naif(data, seuil=0.5):
    pbtg = pbg_continu(data,seuil) * ptg_continu(data)
    pbtp = pbp_continu(data,seuil) * ptp_continu(data)
    pbt = pbtg * pg_continu(data) + pbtp * pp_continu(data)

    p_G = (pg_continu(data) * pbtg) / pbt if pbt > 0 else 0
    p_P = (pp_continu(data) * pbtp) / pbt if pbt > 0 else 0

    if p_G > p_P:
        return f"Nationalité G avec proba MAP={p_G:.3f}"
    elif p_P > p_G:
        return f"Nationalité P avec proba MAP={p_P:.3f}"
    else:
        return f"Indécis (égalité, proba={p_G:.3f})"
print(classer_individu_continu_naif(data_continu))

Nationalité P avec proba MAP=0.624


In [14]:
# Exemple de modification sans naïve Bayes des valeurs continues
def classer_individu_continu_sans_naif(data, seuil=0.5):
    nb_BT_G = sum(1 for intensite,h,p in data if est_blond(intensite,seuil) and h=='T' and p=='G')
    nb_BT_P = sum(1 for intensite,h,p in data if est_blond(intensite,seuil) and h=='T' and p=='P')
    nb_BT = nb_BT_G + nb_BT_P

    p_G = nb_BT_G/nb_BT if nb_BT>0 else 0
    p_P = nb_BT_P/nb_BT if nb_BT>0 else 0

    if p_G > p_P:
        return f"Nationalité G avec proba MAP={p_G:.3f}"
    elif p_P > p_G:
        return f"Nationalité P avec proba MAP={p_P:.3f}"
    else:
        return f"Indécis (égalité, proba={p_G:.3f})"
print(classer_individu_continu_sans_naif(data_continu))

Nationalité P avec proba MAP=0.684


In [15]:
# Exemple du MLE avec naïve Bayes des valeurs continues
def mle_naif_continu(data, seuil=0.5):
    classes = set(p for _,_,p in data)
    vraisemblances = {}
    for c in classes:
        nb_c = sum(1 for _,_,p in data if p==c)
        nb_cheveux_c = sum(1 for intensite,h,p in data if p==c and est_blond(intensite,seuil))
        nb_hauteur_c = sum(1 for intensite,h,p in data if p==c and h=='T')
        p_cheveux = nb_cheveux_c/nb_c if nb_c>0 else 0
        p_hauteur = nb_hauteur_c/nb_c if nb_c>0 else 0
        vraisemblances[c] = p_cheveux * p_hauteur
    best_class = max(vraisemblances, key=vraisemblances.get)
    return best_class, vraisemblances
print("MLE naif:", mle_naif_continu(data_continu))

MLE naif: ('P', {'G': 0.1499375260308205, 'P': 0.23875432525951557})


In [16]:
# Exemple du MLE sans naïve Bayes des valeurs continues
def mle_sans_naif_continu(data, seuil=0.5):
    classes = set(p for _,_,p in data)
    vraisemblances = {}
    for c in classes:
        nb_c = sum(1 for _,_,p in data if p==c)
        nb_obs_c = sum(1 for intensite,h,p in data if p==c and est_blond(intensite,seuil) and h=='T')
        vraisemblances[c] = nb_obs_c/nb_c if nb_c>0 else 0
    best_class = max(vraisemblances, key=vraisemblances.get)
    return best_class, vraisemblances
print("MLE sans naif:", mle_sans_naif_continu(data_continu))

MLE sans naif: ('P', {'G': 0.12244897959183673, 'P': 0.2549019607843137})
